In [1]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [2]:
K = Config()
K.addFile("../rai-robotModels/pr2/pr2.g")
K.addFile("../models/tables.g")

K.addFrame("obj0", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 0 .15)>" )
K.addFrame("obj1", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .2 .15)>" )
K.addFrame("obj2", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .4 .15)>" )
K.addFrame("obj3", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .6.15)>" )
K.addFrame("tray", "table2", "type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:<t(0 0 .07)>" );
K.addFrame("", "tray", "type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]" )
V = K.view()

In [3]:
lgp = K.lgp("../models/fol-pickAndPlace.g")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '<ROOT>',
 'path': '',
 'state': ' STATE_0(START_STATE) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), }'}

In [4]:
lgp.walkToNode("(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray)")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [7]:
lgp.optBound(BT.seq, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0,
  0.0,
  0.12797311332990757,
  0.03784019972400142,
  0.0],
 'boundsCost': [0.30000000000000004,
  0.0,
  0.3145678121207873,
  2.182990977228261,
  0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [6]:
lgp.optBound(BT.path, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0,
  0.0,
  0.12797311332990757,
  0.03784019972400142,
  0.0],
 'boundsCost': [0.30000000000000004,
  0.0,
  0.3145678121207873,
  2.182990977228261,
  0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [4]:
print("THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.")
lgp.addTerminalRule("(on obj0 tray) (on obj1 tray) (on obj2 tray)")
lgp.run(2)

THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.


In [16]:
# wait until you have some number of solutions found (repeat executing this line...)
lgp.numSolutions()

9

In [21]:
# query the optimization features of the 0. solution
lgp.getReport(0, BT.seqPath)

[{'description': 'qItself:ALL',
  'scale': 0.01,
  'sos_value': 312.8296873584427,
  'type': 'sos'},
 {'feature': 'Transition',
  'scale': 1.0,
  'sos_value': 3.5315996716339613,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 3.3731969629743136e-10,
  'type': 'sos'},
 {'description': 'qItself:ALL',
  'scale': 0.1,
  'sos_value': 16.70993245766573,
  'target': [-0.1907157291552083,
   1.335839571564635,
   0.9106530978682812,
   0.10009415734950788,
   0.005149999208797491,
   -0.018120093039719083,
   -0.23104250412240399,
   0.9197021571902088,
   0.4020314565793335,
   0.5547947338128082,
   0.35629529278549016,
   -1.1717250717969212,
   1.0012183699606378,
   -1.4742062832750031,
   -1.9998929855301468,
   -1.5682717081480497,
   1.5024236914990075,
   -0.35794420556149753,
   -0.5158462980543062,
   -0.2721384419308096,
   0.5074533587735766,
   -0.014987605791286203,
   0.011060540287155694,
   -0.08387134660504407,
   0.4331543788741302,
   

In [22]:
# get the KOMO object for the seqPath computation of the 0. solution
komo = lgp.getKOMO(0, BT.seqPath)

In [23]:
komo.displayTrajectory() #SOOO SLOOOW (TODO: add parameter for display speed)

In [14]:
# assign K to the 20. configuration of the 0. solution, check display
# you can now query anything (joint state, frame state, features)
X = komo.getConfiguration(20)
K.setFrameState(X) 

In [20]:
lgp.stop() #stops the thread... takes a while to finish the current job

In [19]:
lgp.run(2) #will continue where it stopped